In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras
import PIL

import tensorflow as tf

Using TensorFlow backend.


In [15]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    patient_age = ds.PatientAge
    scan_part = ds.BodyPartExamined
    position = ds.PatientPosition
    if int(patient_age)>70:
        print(f'WARNING: Patient age is {patient_age} > 70')
    if scan_part != "CHEST":
        print(f'WARNING: CHEST is NOT scanned. Scanned body part is {scan_part}')
    if position not in ("PA", "AP"):
        print(f'WARNING: Patient position is {position}')
        
    print(f"Study Description: {ds.StudyDescription}")
    img = ds.pixel_array
    
    img_gray = PIL.Image.fromarray(img)
    img_RGB = PIL.Image.new("RGB", img_gray.size)
    img_RGB.paste(img_gray)
    img_array = np.asarray(img_RGB, dtype=float)
    
    return img_array
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img, img_mean=[0.485, 0.456, 0.406], img_std=[0.229, 0.224, 0.225],
                     img_size=(224, 224), rescale=1/255): 
    # todo
    
    image_copy = tf.image.resize(img, img_size)
    image_copy = image_copy[25:150, 25:200,:]
    image_copy = rescale*tf.image.resize(image_copy, img_size)
    
    proc_img = (image_copy - img_mean)/img_std
    
    return tf.expand_dims(proc_img, 0)

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    with open(model_path, 'r') as json_file:
        json_model= json_file.read()
        
    model = keras.models.model_from_json(json_model)
    model.load_weights(weight_path)    
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh):
    # todo    
    pred = model.predict(img, steps=1)
    
    prediction = 'Pneumonia'if pred > thresh else 'No Pneumonia'
    
    return f'Prediction: {prediction}\n=================='

In [16]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model_vgg_standardized.json"#path to saved model
weight_path = "{}_my_model_vgg3_standardized.best.hdf5".format('xray_class')#path to saved best weights

IMG_SIZE=(224,224) # This might be different if you did not use vgg16
img_mean = [0.485, 0.456, 0.406] # loads the mean image value they used during training preprocessing
img_std = [0.229, 0.224, 0.225] # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)#loads model
thresh = .35 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
#     img = np.expand_dims(img, 0)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img, img_mean=[0.485, 0.456, 0.406], img_std=[0.229, 0.224, 0.225],
                                img_size=IMG_SIZE, rescale=1/255)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred,'\n')

Load file test1.dcm ...
Study Description: No Finding
Prediction: No Pneumonia

Load file test2.dcm ...
Study Description: Cardiomegaly
Prediction: Pneumonia

Load file test3.dcm ...
Study Description: Effusion
Prediction: Pneumonia

Load file test4.dcm ...
Study Description: No Finding
Prediction: No Pneumonia

Load file test5.dcm ...
Study Description: No Finding
Prediction: No Pneumonia

Load file test6.dcm ...
Study Description: No Finding
Prediction: No Pneumonia



**A note on prediction**
check_dicom will read the important data form DICOM file and compare it with the algorithm requirements such as age, scanned body part, and patient position. If any of the aforementioned requirements denied, instead of rejecting prediction, the patient’s image will be passed onto the model to predict the presence of pneumonia, however, a warning message will also be included in the final report indicating the flaw. To compare the result, Study Description from DICOM file is also printed.